In [1]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, Embedding, Input, Reshape, Concatenate
from keras import metrics
from keras.optimizers import Adam, rmsprop
from keras.callbacks import TensorBoard, ReduceLROnPlateau
from time import time
from IPython.display import display
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
experiment_name='mlhw-mlp-with-preprocess'

## Load Dataset

In [3]:
x_train = pd.read_csv('dataset/train_values.csv')
y_train = pd.read_csv('dataset/train_labels.csv')
x_test = pd.read_csv('dataset/test_values.csv')


# Convert string values to integers
x_train["thal"] = x_train["thal"].astype('category')
x_train["thal"] = x_train["thal"].cat.codes

x_test["thal"] = x_test["thal"].astype('category')
x_test["thal"] = x_test["thal"].cat.codes



# Drop column
x_train = x_train.drop("patient_id", axis=1)
y_train = y_train.drop("patient_id", axis=1)

xsize = x_train.shape[1]
ysize = 1
label_column = 'heart_disease_present'
cols = list(x_train.columns)



In [4]:
display(x_train.head())

,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,1,1,128,2,0,0,2,308,0.0,1,45,170,0
1,2,1,110,3,0,0,0,214,1.6,0,54,158,0
2,1,1,125,4,3,0,2,304,0.0,1,77,162,1
3,1,2,152,4,0,0,0,223,0.0,1,40,181,0
4,3,2,178,1,0,0,2,270,4.2,1,59,145,0


## Preprocessing

### Data Whitening
This will normalize our values so higher order values won't dominate other values

In [5]:
x_train_mean = x_train[cols].mean(axis=0)
x_train_std = x_train[cols].std(axis=0)
x_train[cols] = (x_train[cols] - x_train_mean) / x_train_std

x_test[cols] = (x_test[cols] - x_train_mean) / x_train_std


display(x_train[cols].head())

,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,-0.888762,-0.645259,-0.194652,-1.231340,-0.716404,-0.437019,0.951196,1.115158,-0.900694,0.670152,-1.051032,0.929891,-0.678852
1,0.727169,-0.645259,-1.252825,-0.165757,-0.716404,-0.437019,-1.051322,-0.667915,0.526148,-1.483908,-0.086892,0.386007,-0.678852
2,-0.888762,-0.645259,-0.371014,0.899825,2.378462,-0.437019,0.951196,1.039283,-0.900694,0.670152,2.377024,0.567302,1.464891
3,-0.888762,1.114538,1.216246,0.899825,-0.716404,-0.437019,-1.051322,-0.497195,-0.900694,0.670152,-1.586666,1.428452,-0.678852
4,2.343100,1.114538,2.744719,-2.296923,-0.716404,-0.437019,0.951196,0.394342,2.844768,0.670152,0.448742,-0.203201,-0.678852


### Re-arrange Data for Categorical and Continous Variables

In [6]:

cat_cols = [
    'thal',
    'chest_pain_type',
    'fasting_blood_sugar_gt_120_mg_per_dl',
    'resting_ekg_results',
    'sex',
    'exercise_induced_angina'
]
cont_cols = [
    'slope_of_peak_exercise_st_segment',
    'num_major_vessels',
    'resting_blood_pressure',
    'serum_cholesterol_mg_per_dl',
    'oldpeak_eq_st_depression',
    'age',
    'max_heart_rate_achieved',
]


def preproc_input(input_data):
    input_list = []
    for cat_col in cat_cols:
        input_list.append(input_data[cat_col].values)

    input_list.append(input_data[cont_cols].values)
    return input_list
    

## Define Model 

In [7]:
initial_lr = 0.01
opt_name = 'adam'
# opt_name = 'rmsprop'

def create_model():
    inputs = []
    embeddings = []

    # For Categorical Variables
    for cat_col in cat_cols :

        no_of_unique_cat  = x_train[cat_col].nunique()
        embedding_size = min(np.ceil((no_of_unique_cat)/2), 50)
        embedding_size = int(embedding_size)
        vocab  = no_of_unique_cat + 1

        emb_input = Input(shape=(1,))
        emb = Embedding(vocab, embedding_size, input_length = 1)(emb_input)
        emb = Dropout(.01)(emb)
        emb = Reshape(target_shape=(embedding_size, ))(emb)
        inputs.append(emb_input)
        embeddings.append(emb)

    # Continuous variables
    input_cont = Input(shape=(len(cont_cols),))
    embedding_cont = BatchNormalization()(input_cont) 
    inputs.append(input_cont)
    embeddings.append(embedding_cont)

    x = Concatenate()(embeddings)
    x = Dense(100)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(.15)(x)

    x = Dense(50)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(.15)(x)

    x = Dense(20)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(.15)(x)

    x = Dense(1)(x)
    output = Activation('sigmoid')(x)

    model = Model(
        inputs=inputs, 
        outputs=[output]
    )


#     opt = rmsprop(lr=initial_lr, decay=1e-6)

    opt = Adam(lr=initial_lr)


    model.compile(
        loss='binary_crossentropy',
        optimizer=opt,
        metrics=[metrics.binary_accuracy]
    )
    return model

## Training

### K-Fold Validation

In [13]:
epochs = 100
batch_size = 32
K = 10
loss_list=[]
acc_list=[]
pred_list = np.zeros((np.shape(x_test)[0],K))

x_test_f = preproc_input(x_test)


kfold = StratifiedKFold(n_splits = K, 
                            random_state = 1, 
                            shuffle = True)  
for i, (train_idxs, val_indxs) in enumerate(kfold.split(x_train.values, y_train.values)):
    print('\nFold ', i + 1)
    x_train_f = x_train.loc[train_idxs]
    y_train_f = y_train.loc[train_idxs]

    x_val_f = x_train.loc[val_indxs]
    y_val_f = y_train.loc[val_indxs]
    
    
    x_train_f = preproc_input(x_train_f)
    x_val_f = preproc_input(x_val_f)
    
    
    model = create_model()
    model.fit(
        x_train_f,
        y_train_f.values,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_val_f, y_val_f),
        verbose=1,
    )
    model_eval = model.evaluate(x_val_f, y_val_f)
    loss_list.append(model_eval[0])
    acc_list.append(model_eval[1])
    pred_list[:, i] += model.predict(x_test_f)[:,0]
    
mean_loss = np.mean(loss_list)
mean_acc = np.mean(acc_list)
mean_pred = np.mean(pred_list, axis = 1)
print("Mean loss: {}".format(mean_loss))
print("Mean acc: {}".format(mean_acc))
print("Mean pred: {}".format(mean_pred))

## Save model and weights
# save_dir=os.path.join(os.getcwd(), 'saved_models')
# model_name= '{}.h5'.format(exp_stamp)
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
# model_path = os.path.join(save_dir, model_name)
# model.save(model_path)
# print('Saved  trained model at %s ' % model_path)


Fold  1
Train on 162 samples, validate on 18 samples
Epoch 1/100
162/162 [==============================] - 2s 12ms/step - loss: 0.6887 - binary_accuracy: 0.6049 - val_loss: 1.1168 - val_binary_accuracy: 0.6667
Epoch 2/100
162/162 [==============================] - 0s 305us/step - loss: 0.5547 - binary_accuracy: 0.7469 - val_loss: 1.1211 - val_binary_accuracy: 0.7222
Epoch 3/100
162/162 [==============================] - 0s 339us/step - loss: 0.4501 - binary_accuracy: 0.7778 - val_loss: 0.8716 - val_binary_accuracy: 0.6667
Epoch 4/100
162/162 [==============================] - 0s 299us/step - loss: 0.4882 - binary_accuracy: 0.7593 - val_loss: 0.6592 - val_binary_accuracy: 0.6667
Epoch 5/100
162/162 [==============================] - 0s 306us/step - loss: 0.4593 - binary_accuracy: 0.8086 - val_loss: 0.7046 - val_binary_accuracy: 0.6667
Epoch 6/100
162/162 [==============================] - 0s 297us/step - loss: 0.4006 - binary_accuracy: 0.8395 - val_loss: 0.7143 - val_binary_accuracy: 

Epoch 52/100
162/162 [==============================] - 0s 280us/step - loss: 0.1661 - binary_accuracy: 0.9506 - val_loss: 0.9763 - val_binary_accuracy: 0.6667
Epoch 53/100
162/162 [==============================] - 0s 255us/step - loss: 0.2521 - binary_accuracy: 0.9012 - val_loss: 0.8462 - val_binary_accuracy: 0.6667
Epoch 54/100
162/162 [==============================] - 0s 266us/step - loss: 0.2289 - binary_accuracy: 0.9074 - val_loss: 0.7529 - val_binary_accuracy: 0.7222
Epoch 55/100
162/162 [==============================] - 0s 273us/step - loss: 0.2131 - binary_accuracy: 0.9321 - val_loss: 0.7316 - val_binary_accuracy: 0.7778
Epoch 56/100
162/162 [==============================] - 0s 267us/step - loss: 0.1967 - binary_accuracy: 0.9383 - val_loss: 0.7416 - val_binary_accuracy: 0.7222
Epoch 57/100
162/162 [==============================] - 0s 271us/step - loss: 0.1857 - binary_accuracy: 0.9321 - val_loss: 0.7034 - val_binary_accuracy: 0.7778
Epoch 58/100
162/162 [==================

162/162 [==============================] - 0s 256us/step - loss: 0.2185 - binary_accuracy: 0.9259 - val_loss: 0.7056 - val_binary_accuracy: 0.8333
Epoch 54/100
162/162 [==============================] - 0s 244us/step - loss: 0.2068 - binary_accuracy: 0.9198 - val_loss: 0.7128 - val_binary_accuracy: 0.7778
Epoch 55/100
162/162 [==============================] - 0s 292us/step - loss: 0.2215 - binary_accuracy: 0.9198 - val_loss: 0.7788 - val_binary_accuracy: 0.7778
Epoch 56/100
162/162 [==============================] - 0s 288us/step - loss: 0.2048 - binary_accuracy: 0.9074 - val_loss: 0.7584 - val_binary_accuracy: 0.8333
Epoch 57/100
162/162 [==============================] - 0s 261us/step - loss: 0.1712 - binary_accuracy: 0.9444 - val_loss: 0.7561 - val_binary_accuracy: 0.7778
Epoch 58/100
162/162 [==============================] - 0s 244us/step - loss: 0.1555 - binary_accuracy: 0.9568 - val_loss: 0.7374 - val_binary_accuracy: 0.7778
Epoch 59/100
162/162 [==============================]

162/162 [==============================] - 0s 268us/step - loss: 0.2038 - binary_accuracy: 0.9136 - val_loss: 0.8381 - val_binary_accuracy: 0.7222
Epoch 55/100
162/162 [==============================] - 0s 268us/step - loss: 0.2115 - binary_accuracy: 0.9321 - val_loss: 0.8777 - val_binary_accuracy: 0.6667
Epoch 56/100
162/162 [==============================] - 0s 248us/step - loss: 0.1993 - binary_accuracy: 0.9321 - val_loss: 0.8634 - val_binary_accuracy: 0.7222
Epoch 57/100
162/162 [==============================] - 0s 287us/step - loss: 0.1764 - binary_accuracy: 0.9383 - val_loss: 0.8424 - val_binary_accuracy: 0.7222
Epoch 58/100
162/162 [==============================] - 0s 285us/step - loss: 0.1961 - binary_accuracy: 0.9074 - val_loss: 0.8544 - val_binary_accuracy: 0.7222
Epoch 59/100
162/162 [==============================] - 0s 268us/step - loss: 0.1431 - binary_accuracy: 0.9753 - val_loss: 0.9295 - val_binary_accuracy: 0.6667
Epoch 60/100
162/162 [==============================]

162/162 [==============================] - 0s 311us/step - loss: 0.2094 - binary_accuracy: 0.9321 - val_loss: 0.6512 - val_binary_accuracy: 0.7222
Epoch 56/100
162/162 [==============================] - 0s 250us/step - loss: 0.2204 - binary_accuracy: 0.9136 - val_loss: 0.6431 - val_binary_accuracy: 0.6667
Epoch 57/100
162/162 [==============================] - 0s 249us/step - loss: 0.1680 - binary_accuracy: 0.9506 - val_loss: 0.6861 - val_binary_accuracy: 0.7222
Epoch 58/100
162/162 [==============================] - 0s 276us/step - loss: 0.1612 - binary_accuracy: 0.9444 - val_loss: 0.7405 - val_binary_accuracy: 0.7222
Epoch 59/100
162/162 [==============================] - 0s 291us/step - loss: 0.1524 - binary_accuracy: 0.9568 - val_loss: 0.8059 - val_binary_accuracy: 0.6667
Epoch 60/100
162/162 [==============================] - 0s 274us/step - loss: 0.1446 - binary_accuracy: 0.9630 - val_loss: 0.8305 - val_binary_accuracy: 0.6111
Epoch 61/100
162/162 [==============================]

...

162/162 [==============================] - 0s 273us/step - loss: 0.1728 - binary_accuracy: 0.9444 - val_loss: 0.5108 - val_binary_accuracy: 0.8333
Epoch 58/100
162/162 [==============================] - 0s 293us/step - loss: 0.2138 - binary_accuracy: 0.9259 - val_loss: 0.4706 - val_binary_accuracy: 0.8333
Epoch 59/100
162/162 [==============================] - 0s 290us/step - loss: 0.1625 - binary_accuracy: 0.9321 - val_loss: 0.4226 - val_binary_accuracy: 0.8333
Epoch 60/100
162/162 [==============================] - 0s 251us/step - loss: 0.1543 - binary_accuracy: 0.9444 - val_loss: 0.3855 - val_binary_accuracy: 0.8333
Epoch 61/100
162/162 [==============================] - 0s 263us/step - loss: 0.1184 - binary_accuracy: 0.9568 - val_loss: 0.3974 - val_binary_accuracy: 0.8333
Epoch 62/100
162/162 [==============================] - 0s 294us/step - loss: 0.1429 - binary_accuracy: 0.9568 - val_loss: 0.4244 - val_binary_accuracy: 0.8333
Epoch 63/100
162/162 [==============================]

162/162 [==============================] - 0s 265us/step - loss: 0.2123 - binary_accuracy: 0.9321 - val_loss: 0.3323 - val_binary_accuracy: 0.8889
Epoch 59/100
162/162 [==============================] - 0s 289us/step - loss: 0.1714 - binary_accuracy: 0.9506 - val_loss: 0.3575 - val_binary_accuracy: 0.7778
Epoch 60/100
162/162 [==============================] - 0s 250us/step - loss: 0.2003 - binary_accuracy: 0.9136 - val_loss: 0.3993 - val_binary_accuracy: 0.7778
Epoch 61/100
162/162 [==============================] - 0s 246us/step - loss: 0.2014 - binary_accuracy: 0.9198 - val_loss: 0.4648 - val_binary_accuracy: 0.7778
Epoch 62/100
162/162 [==============================] - 0s 264us/step - loss: 0.1982 - binary_accuracy: 0.9259 - val_loss: 0.5759 - val_binary_accuracy: 0.6667
Epoch 63/100
162/162 [==============================] - 0s 260us/step - loss: 0.2395 - binary_accuracy: 0.9259 - val_loss: 0.5331 - val_binary_accuracy: 0.6667
Epoch 64/100
162/162 [==============================]

Epoch 9/100
162/162 [==============================] - 0s 201us/step - loss: 0.3421 - binary_accuracy: 0.8642 - val_loss: 0.5416 - val_binary_accuracy: 0.7222
Epoch 10/100
162/162 [==============================] - 0s 209us/step - loss: 0.3226 - binary_accuracy: 0.8827 - val_loss: 0.5111 - val_binary_accuracy: 0.7222
Epoch 11/100
162/162 [==============================] - 0s 217us/step - loss: 0.2909 - binary_accuracy: 0.9012 - val_loss: 0.5531 - val_binary_accuracy: 0.7222
Epoch 12/100
162/162 [==============================] - 0s 253us/step - loss: 0.3082 - binary_accuracy: 0.8704 - val_loss: 0.5666 - val_binary_accuracy: 0.7222
Epoch 13/100
162/162 [==============================] - 0s 254us/step - loss: 0.2687 - binary_accuracy: 0.8827 - val_loss: 0.5615 - val_binary_accuracy: 0.7222
Epoch 14/100
162/162 [==============================] - 0s 245us/step - loss: 0.2787 - binary_accuracy: 0.8889 - val_loss: 0.5219 - val_binary_accuracy: 0.7222
Epoch 15/100
162/162 [===================

Epoch 11/100
162/162 [==============================] - 0s 272us/step - loss: 0.3311 - binary_accuracy: 0.8642 - val_loss: 0.7658 - val_binary_accuracy: 0.7222
Epoch 12/100
162/162 [==============================] - 0s 281us/step - loss: 0.3202 - binary_accuracy: 0.8951 - val_loss: 0.8543 - val_binary_accuracy: 0.7222
Epoch 13/100
162/162 [==============================] - 0s 249us/step - loss: 0.3575 - binary_accuracy: 0.8704 - val_loss: 0.8155 - val_binary_accuracy: 0.6667
Epoch 14/100
162/162 [==============================] - 0s 246us/step - loss: 0.3411 - binary_accuracy: 0.8704 - val_loss: 0.8052 - val_binary_accuracy: 0.6667
Epoch 15/100
162/162 [==============================] - 0s 234us/step - loss: 0.3137 - binary_accuracy: 0.8827 - val_loss: 0.7593 - val_binary_accuracy: 0.7222
Epoch 16/100
162/162 [==============================] - 0s 268us/step - loss: 0.2854 - binary_accuracy: 0.8951 - val_loss: 0.7448 - val_binary_accuracy: 0.7222
Epoch 17/100
162/162 [==================

Epoch 13/100
162/162 [==============================] - 0s 250us/step - loss: 0.3467 - binary_accuracy: 0.8333 - val_loss: 0.6869 - val_binary_accuracy: 0.7222
Epoch 14/100
162/162 [==============================] - 0s 257us/step - loss: 0.3890 - binary_accuracy: 0.8210 - val_loss: 0.6965 - val_binary_accuracy: 0.7222
Epoch 15/100
162/162 [==============================] - 0s 258us/step - loss: 0.3223 - binary_accuracy: 0.8642 - val_loss: 0.6952 - val_binary_accuracy: 0.7222
Epoch 16/100
162/162 [==============================] - 0s 261us/step - loss: 0.3153 - binary_accuracy: 0.8951 - val_loss: 0.6286 - val_binary_accuracy: 0.7778
Epoch 17/100
162/162 [==============================] - 0s 226us/step - loss: 0.2944 - binary_accuracy: 0.8951 - val_loss: 0.6249 - val_binary_accuracy: 0.7778
Epoch 18/100
162/162 [==============================] - 0s 240us/step - loss: 0.2933 - binary_accuracy: 0.8765 - val_loss: 0.6376 - val_binary_accuracy: 0.7222
Epoch 19/100
162/162 [==================

## Write output to File

In [17]:
out_df = pd.DataFrame({
    'patient_id': x_test['patient_id'],
    'heart_disease_present': mean_pred[:]
})
out_df = out_df[['patient_id', 'heart_disease_present']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(out_df)

out_dir = 'output'
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
out_file = out_dir + '/' + experiment_name + '-' + 'out.csv'
out_df.to_csv(out_file, index=False)

,patient_id,heart_disease_present
0,olalu7,0.447493
1,z9n6mx,0.028735
2,5k4413,0.947164
3,mrg7q5,0.147288
4,uki4do,0.931206
5,kev1sk,0.035303
6,9n6let,0.118551
7,jxmtyg,0.969337
8,51s2ff,0.273389
9,wi9mcs,0.148902
